## Домашнее задание №6 к лекции «Исключения и обработка ошибок»

In [1]:
from datetime import datetime, timedelta
import os

### Задание 1
Печатные газеты использовали свой формат дат для каждого выпуска. Для каждой газеты из списка напишите формат указанной даты для перевода в объект datetime:

The Moscow Times - Wednesday, October 2, 2002  
The Guardian - Friday, 11.10.13  
Daily News - Thursday, 18 August 1977  

In [2]:
date = datetime(2002, 10, 2, 0, 0, 0, 0)

Тут можно было бы и попроще, но решил попрактиковаться с зипом и словарями

In [3]:
newspapers = ['the_moscow_times', 'the_guardian', 'daily_news']
date_formats = ['%A, %B %-d,%Y', '%A, %d.%m.%y', '%A, %d %B %Y']
for newspaper,date_format in (dict(zip(newspapers, date_formats))).items():
    print(newspaper.title().replace('_',' '), '-', date.strftime(date_format))

The Moscow Times - Wednesday, October 2,2002
The Guardian - Wednesday, 02.10.02
Daily News - Wednesday, 02 October 2002


### Задание 2
Дан поток дат в формате YYYY-MM-DD, в которых встречаются некорректные значения:
stream = [‘2018-04-02’, ‘2018-02-29’, ‘2018-19-02’]

Напишите функцию, которая проверяет эти даты на корректность. Т. е. для каждой даты возвращает True (дата корректна) или False (некорректная дата).

In [4]:
stream = ['2018-04-02', '2018-02-29', '2018-19-02']

Это если функция применяется к конкретной дате в списке

In [5]:
def date_validator_1(date):
    try:
        datetime.strptime(date, '%Y-%m-%d')
        return True
    except:
        return False

In [6]:
for date in stream:
    print(date_validator_1(date))

True
False
False


Это если функция применяется ко всему списку

In [7]:
def date_validator_2(date_lst):
    for date in date_lst:
        try:
            datetime.strptime(date, '%Y-%m-%d')
            print(date, True)
        except:
            print(date, False)

In [8]:
date_validator_2(stream)

2018-04-02 True
2018-02-29 False
2018-19-02 False


Это если надо получить список списков. Может понадобится его куда-нибудь залить. Словарь наверно не очень удачная идея, т.к. даты в потоке могут быть не уникальные, а при `setdefault` первоначальное значение `True` или `False` не изменится

In [9]:
def date_validator_3(date_lst):
    result = []
    for date in date_lst:
        try:
            datetime.strptime(date, '%Y-%m-%d')
            result.append([date, True])
        except:
            result.append([date, False])
    return result

In [10]:
date_validator_3(stream)

[['2018-04-02', True], ['2018-02-29', False], ['2018-19-02', False]]

Это на случай если нам этот результат понадобится сохранить в нужной папке текущего проекта в csv

In [11]:
def date_validator_4(date_lst, path):
    cnt_err = 0
    with open(path + 'validator.csv', 'w') as f:
        for date in date_lst:
            try:
                datetime.strptime(date, '%Y-%m-%d')
                f.write(date + ',')
                f.write(str(True) + '\n')
            except:
                cnt_err += 1
                f.write(date + ',')
                f.write(str(False) + '\n')
    print(f'Список обработан и схранен в validator.csv, \
    \nзагружено строк - {len(date_lst)},\
    \nстрок с ошибками - {cnt_err},\
    \nпроцент ошибок - {cnt_err/len(date_lst):.0%}')

In [12]:
path = os.getcwd()+'/Downloads/' # тут получаем путь до папки с проектом и добавляем папку куда класть результат

In [13]:
date_validator_4(stream, path)

Список обработан и схранен в validator.csv,     
загружено строк - 3,    
строк с ошибками - 2,    
процент ошибок - 67%


### Задание 3  
Напишите функцию date_range, которая возвращает список дат за период от start_date до end_date. Даты должны вводиться в формате YYYY-MM-DD. В случае неверного формата или при start_date > end_date должен возвращаться пустой список.

В этой функции есть смысл применить функцию из предыдущего задания `date_validator_1`

In [14]:
def date_range(start_date, end_date):
    date_format = '%Y-%m-%d'
    result = []
    if date_validator_1(start_date) and date_validator_1(end_date) and start_date < end_date:
        sd = datetime.strptime(start_date, date_format)
        ed = datetime.strptime(end_date, date_format)
        date_diff = (ed-sd).days
        for i in range(date_diff + 1):
            result.append((sd + timedelta(days=i)).strftime(date_format))
    return result

In [15]:
sd = '2018-01-29'
ed = '2018-02-11'
date_range(sd, ed)

['2018-01-29',
 '2018-01-30',
 '2018-01-31',
 '2018-02-01',
 '2018-02-02',
 '2018-02-03',
 '2018-02-04',
 '2018-02-05',
 '2018-02-06',
 '2018-02-07',
 '2018-02-08',
 '2018-02-09',
 '2018-02-10',
 '2018-02-11']

In [16]:
sd = '2018-05-29'
ed = '2018-02-02'
date_range(sd, ed)

[]

In [17]:
sd = '2018-33-56'
ed = '2018-02-02'
date_range(sd, ed)

[]

### Задание 4 (необязательное)
Ваш коллега прислал код функции:
При однократном вызове этой функции все работает корректно:

delete_and_return_last_user(1)
‘A101’

Однако, при повторном вызове получается ошибка IndexError: list index out of range.

Задание:

Что значит ошибка list index out of range?
Почему при первом запуске функция работает корректно, а при втором - нет?

In [18]:
def delete_and_return_last_user(region, default_list=['A100', 'A101', 'A102']):
    """
    Удаляет из списка default_list последнего пользователя
    и возвращает ID нового последнего пользователя.
    """
    element_to_delete = default_list[-1]
    default_list.remove(element_to_delete)
    print(element_to_delete)
    print(default_list)
    print(DEFAULT_USER_COUNT-2)
    return default_list[DEFAULT_USER_COUNT-2]

In [19]:
DEFAULT_USER_COUNT = 3

In [20]:
delete_and_return_last_user(1)

A102
['A100', 'A101']
1


'A101'

Распишем последовательно все ходы:  
1. При первом вызове индексы __default_list__  0,1,2. а значения ['A100', 'A101', 'A102']
2. Списки изменяемые объекты. Удаляем последний индекс, __default_list__ становится  ['A100', 'A101'] а индексы становятся 0,1. Индекс в ретурне 3-2 = 1. на вывод A101
3. При повторном запуске __default_list__ остается ['A100', 'A101'] с индексами 0,1. Мы так же удаляем последний элемент, __default_list__ становится из одного элемента ['A100'] с индексом 0
4. Т.к функция ссылается на внешнюю переменную __DEFAULT_USER_COUNT__ = 3 ретурн снова пытается получить значение с индексом 1, а в списке __default_list__ такого индекса уже нет. следовательно ошибка